# Objective:
Notebook to generate index for the table/dataframe


# Imports

In [2]:
import pandas as pd
import pickle
import os
import re
import json
import importlib
from typing import Optional
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT, ID
from whoosh.qparser import QueryParser, FuzzyTermPlugin
DATA_PATH = '/Users/kumarp05/table_talks/data/'

# Read the file    

In [60]:
def read_file(file_name: str) -> Optional[pd.DataFrame]:
    """Function to read different formats of spreadsheets like CSV, XLS, XLSX, and Parquet based on the extension of the file uploaded

    Args:
        file_name (str): The name of the spreadsheet file to be read

    Returns:
        Optional[pd.DataFrame]: A DataFrame object containing the spreadsheet data, or None if the file format is not supported

    Raises:
        NotImplementedError: If an unsupported file format is passed as a parameter"""

    # Determine the file format from its extension
    file_ext = file_name.split(".")[-1].lower()

    if file_ext == "csv":
        df = pd.read_csv(file_name)
    elif file_ext == "xls" or file_ext == "xlsx":
        df = pd.read_excel(file_name)
    elif file_ext == "parquet":
        df = pd.read_parquet(file_name)
    else:
        print(f"Unsupported file format: {file_ext}")
        return None

    # Return the DataFrame containing the spreadsheet data
    return df
    
file_name = 'core_hvp_diff.xlsx'
df = read_file(file_name)
df.head()

,Group,GLOBAL CUSTOMER FULL,ActualNetSales_sum,ActualQty_sum,GroupContributionToOurRevenue%,UnitPrice,UnitPriceDiff,created_on,billed_on
0,Core Products,NOVO NORDISK AS,98086461.33,5000494.410,84.476,19.615353,NaN,"Thursday, 12 January 2023",2000-01-01
1,HVP,NOVO NORDISK AS,18025645.78,494998.814,15.524,36.415533,16.800181,"Friday, 13 January 2023",2000-01-02
2,Core Products,INIBSA,12677914.26,1129776.959,96.992,11.221608,NaN,"Saturday, 14 January 2023",2000-01-03
3,HVP,INIBSA,393157.99,10450.237,3.008,37.621921,26.400313,"Sunday, 15 January 2023",2000-01-04
4,Core Products,YICHANG RENFU MEDICINE,581579.18,29382.000,98.681,19.793723,NaN,"Monday, 16 January 2023",2000-01-05


# Getting Dtypes of the table:
- TODO: Automatic inferring
- TODO: Manual changing option of dtypes of columns

In [65]:
def get_dtypes(df: pd.DataFrame) -> dict[str, type]:
    """Function to get the data types of each column in a DataFrame and store them in a dictionary with column name as key and dtype as value

    Args:
        df (pd.DataFrame): The Pandas DataFrame object for which we want to get the data types

    Returns:
        dict[str, type]: A dictionary containing the column names and corresponding data types of the input DataFrame
    """

    # Get the dtypes of each column in the input DataFrame
    column_dtypes = df.dtypes.to_dict()

    # Rename the keys to be the column names instead of the dtype names
    col_name_dtypes = {k: v for k, v in column_dtypes.items()}

    # Return the dictionary containing the data types of each column
    return col_name_dtypes

data_type = get_dtypes(df)
print(data_type)

{'Group': dtype('O'), 'GLOBAL CUSTOMER FULL': dtype('O'), 'ActualNetSales_sum': dtype('float64'), 'ActualQty_sum': dtype('float64'), 'GroupContributionToOurRevenue%': dtype('float64'), 'UnitPrice': dtype('float64'), 'UnitPriceDiff': dtype('float64'), 'created_on': dtype('O'), 'billed_on': dtype('<M8[ns]')}


# Extracting Unique Values
- Of Categorical Columns
- TODO: Option to avoid indexing ID columns

In [62]:
import pandas as pd

def get_unique_values(df: pd.DataFrame, cat_cols: list) -> dict[str, list]:
    """Function to return all the unique values of a given set of columns in a DataFrame

    Args:
        df (pd.DataFrame): The Pandas DataFrame object for which we want to get the unique values
        cat_cols (str): A string containing a comma-separated list of column names for which we want to get the unique values

    Returns:
        dict[str, list]: A dictionary containing the unique values of each specified column as a list

    Raises:
        ValueError: If any of the specified columns do not exist in the DataFrame
    """

    # Create an empty dictionary to store the unique values of each column
    unique_values = {}

    # Iterate over each specified column and get its unique values
    for col in cat_cols:
        if col not in df.columns:
            raise ValueError(f"Column '{col}' does not exist in DataFrame")
        else:
            unique_values[col] = df[col].unique().tolist()

    # Return the dictionary containing the unique values of each column
    return unique_values

get_unique_values(df, ['Group','GLOBAL CUSTOMER FULL'])

{'Group': ['Core Products', 'HVP'],
 'GLOBAL CUSTOMER FULL': ['NOVO NORDISK AS',
  'INIBSA',
  'YICHANG RENFU MEDICINE',
  'BAXTER INTERNATIONAL',
  'BECTON DICKINSON & COMPANY',
  'HUONS - KOREA',
  'B.BRAUN MELSUNGEN AG',
  'REIG JOFRE GROUP',
  'JSC FARMAK',
  'SQUARE PHARMACEUTICALS LIMITED',
  'LABORATORIO P. CASSARA S.R.L.',
  'LYOCONTRACT - GLOBAL',
  'ISO-TEX DIAGNOSTICS INC',
  'VALDEPHARM',
  'DWK LIFE SCIENCES LIMITED',
  'TRICORBRAUN',
  'APSEN FARMACEUTICA S/A',
  'SMALL VOLUME - CORE',
  'LG CHEMICAL - KOREA',
  'EUROFARMA',
  'GEN ILAC VE SAGLIK √úR√úNLERI AS.',
  'SINTETICA SA',
  'CURIUM US LLC',
  'NEON LABORATORIES LTD',
  'ZOETIS',
  'SIEMENS AG',
  'PT SINAR GOLDSINDO',
  'EVAPHARMA FOR PHARMACEUTICAL',
  'G.S. COSMECEUTICAL USA INC',
  'CARDINAL HEALTH',
  'INDUSTRIAL VETERINARIA, S.A.',
  'GE HEALTHCARE',
  'WPS AFFILIATES',
  'ALCON RESEARCH LLC',
  'GRIFOLS INC',
  'STELIS BIOPHARMA LIMITED',
  'NCPC BIOTECH SUPPLIES BRANCH',
  'EMAD TRADE HOUSE FZCO',
  'RECIP

# Getting Categorical Columns
- From data_types

In [66]:
def get_cat_cols(data_types: dict) -> list[str]:
    """Function to return all the categorical column names from a schema dictionary

    Args:
        data_types (dict): A dictionary containing column names as keys and their corresponding data types as values

    Returns:
        list[str]: A list of categorical column names in the input schema dictionary
    """

    # Create an empty list to store the categorical column names
    cat_cols = []

    # Iterate over each column in the input schema dictionary and check if it is a categorical column or not
    for col, dtype in data_types.items():
        if dtype == 'object':
            cat_cols.append(col)
        elif dtype.name == 'category':
            cat_cols.append(col)
        # Add more conditions to check if a data type is categorical or not depending on the use case

    # Return the list of categorical column names
    return cat_cols

# Generating a dictionary having all unique values of all the categorical columns
categorical_columns = get_unique_values(df=df, cat_cols=get_cat_cols(data_types=data_type))
categorical_columns

{'Group': ['Core Products', 'HVP'],
 'GLOBAL CUSTOMER FULL': ['NOVO NORDISK AS',
  'INIBSA',
  'YICHANG RENFU MEDICINE',
  'BAXTER INTERNATIONAL',
  'BECTON DICKINSON & COMPANY',
  'HUONS - KOREA',
  'B.BRAUN MELSUNGEN AG',
  'REIG JOFRE GROUP',
  'JSC FARMAK',
  'SQUARE PHARMACEUTICALS LIMITED',
  'LABORATORIO P. CASSARA S.R.L.',
  'LYOCONTRACT - GLOBAL',
  'ISO-TEX DIAGNOSTICS INC',
  'VALDEPHARM',
  'DWK LIFE SCIENCES LIMITED',
  'TRICORBRAUN',
  'APSEN FARMACEUTICA S/A',
  'SMALL VOLUME - CORE',
  'LG CHEMICAL - KOREA',
  'EUROFARMA',
  'GEN ILAC VE SAGLIK √úR√úNLERI AS.',
  'SINTETICA SA',
  'CURIUM US LLC',
  'NEON LABORATORIES LTD',
  'ZOETIS',
  'SIEMENS AG',
  'PT SINAR GOLDSINDO',
  'EVAPHARMA FOR PHARMACEUTICAL',
  'G.S. COSMECEUTICAL USA INC',
  'CARDINAL HEALTH',
  'INDUSTRIAL VETERINARIA, S.A.',
  'GE HEALTHCARE',
  'WPS AFFILIATES',
  'ALCON RESEARCH LLC',
  'GRIFOLS INC',
  'STELIS BIOPHARMA LIMITED',
  'NCPC BIOTECH SUPPLIES BRANCH',
  'EMAD TRADE HOUSE FZCO',
  'RECIP

# Synonyms
- Simple implementation of plural variations
- TODO: Generating relevant variation of synonyms using llm

In [75]:
import re

def split_and_join_string(s):
    # Splitting snake_case
    if '_' in s:
        split_words = s.split('_')
    
    # Splitting camelCase and PascalCase
    else:
        split_words = re.findall('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', s)
    
    # Joining the split words with a space
    return ' '.join(split_words)

# Examples
print(split_and_join_string("camelCaseExample"))  # camelCase
print(split_and_join_string("PascalCaseExample")) # PascalCase
print(split_and_join_string("snake_case_example")) # snake_case
print(split_and_join_string("UnitPrice")) # snake_case



camel Case Example
Pascal Case Example
snake case example
Unit Price


In [ ]:
def create_synonym_dict(data_types: dict, generate_synonyms: bool = False) -> dict:
    """
    Creates a dictionary with keys as Attributes and their plural forms as basic synonyms.

    Args:
         data_types: A dictionary containing attribute names as keys.

    Returns: 
        A dictionary with attribute names as keys and their plural forms as values.
    """
    
    synonyms = {}
    for column_name in data_types.keys():

        # Split and create a single word in case of camelCase, PascalCase or snake_case for better search results
        split_col_name = split_and_join_string(column_name)
        synonyms[column_name] = [split_col_name]

        if column_name[-1] == "y":
            synonyms[column_name].append(f"{split_col_name[:-1]}ies")
        elif column_name[-1] in ("s", "x"):
            synonyms[column_name].append(f"{split_col_name[:-1]}es")
        else:
            synonyms[column_name].append(f"{split_col_name}s")
        
        # Appending generated synonyms
        if generate_synonyms:
            try:
                synonyms[column_name].extend(eval(generate_prompt.generate_synonyms([split_col_name])))
                # In case LLM output is not a list
            except Exception as e:
                print(f"Error in Generating Synonyms: {e}")
                synonyms[column_name].extend(generate_prompt.generate_synonyms([split_col_name]))

    return synonyms

synonyms = create_synonym_dict(data_types=data_type, generate_synonyms=True)
synonyms

# Persist Schema

In [79]:
kb_dict = {'data_type':data_type,'categorical_columns':categorical_columns,'synonyms':synonyms}
with open(f"{file_name}.pkl", 'wb') as f:
    pickle.dump(kb_dict, f)

# # Persist in JSON
# with open(f'{file_name}.json', 'w') as f:
#     json.dump(kb_dict, f , indent=4)

In [80]:
with open(f"{file_name}.pkl", 'rb') as f:
    schema_dict = pickle.load(f)

# Indexing Knowledgebase

In [81]:
# Decalaring Index Schema
 # Define the schema for the index 
schema = Schema(column_name=ID(stored=True), synonyms=TEXT(stored=True), unique_values=TEXT(stored=True))  

# Create an index directory if it doesn't exist  
index_path = f"{file_name}_kb_index"
if not os.path.exists(index_path):  
    os.mkdir(index_path)  
  
# Create the index  
ix = create_in(index_path, schema)  
  
writer = ix.writer()  
  
for k,v in schema_dict['data_type'].items():

      column_name = f"'{k}', '{v}'" 
      print('column_name:', column_name)

      # If it's Edm.DateTime store in 'synonyms' field
      if 'datetime' in str(v):
        synonyms = f"{str(schema_dict['synonyms'][k])}, {schema_dict['data_type'][k]}"
        print('synonyms:', synonyms)
      else:
        synonyms = str(schema_dict['synonyms'][k])
        print('synonyms:', synonyms)
      # Catch exception in unique_values as only categorical columns are present in it
      try:
        unique_values = str(schema_dict['categorical_columns'][k])
        print("unique_values:", unique_values)
      except:
        unique_values = ''
      
      writer.add_document(column_name=column_name, synonyms=synonyms, unique_values=unique_values)
  
writer.commit()

column_name: 'Group', 'object'
synonyms: ['Group', 'Groups', 'Team', 'Class', 'Cluster', 'Cohort', 'Squad', 'Section', 'Division', 'Guild', 'Circle', 'Collective']
unique_values: ['Core Products', 'HVP']
column_name: 'GLOBAL CUSTOMER FULL', 'object'
synonyms: ['GLOBAL CUSTOMER FULL', 'GLOBAL CUSTOMER FULLs', 'Client', 'Consumer', 'Patron', 'User', 'CustomerBase', 'Market', 'Audience', 'Demographic', 'Public', 'Community']
unique_values: ['NOVO NORDISK AS', 'INIBSA', 'YICHANG RENFU MEDICINE', 'BAXTER INTERNATIONAL', 'BECTON DICKINSON & COMPANY', 'HUONS - KOREA', 'B.BRAUN MELSUNGEN AG', 'REIG JOFRE GROUP', 'JSC FARMAK', 'SQUARE PHARMACEUTICALS LIMITED', 'LABORATORIO P. CASSARA S.R.L.', 'LYOCONTRACT - GLOBAL', 'ISO-TEX DIAGNOSTICS INC', 'VALDEPHARM', 'DWK LIFE SCIENCES LIMITED', 'TRICORBRAUN', 'APSEN FARMACEUTICA S/A', 'SMALL VOLUME - CORE', 'LG CHEMICAL - KOREA', 'EUROFARMA', 'GEN ILAC VE SAGLIK √úR√úNLERI AS.', 'SINTETICA SA', 'CURIUM US LLC', 'NEON LABORATORIES LTD', 'ZOETIS', 'SIEMENS

# Consuming Index

In [51]:
# Load Index
file_name = 'jobs_in_data.csv'
from whoosh.index import open_dir
ix = open_dir(f"{file_name}_kb_index")

In [57]:
def identify_column_from_value(ix, keyword, use_wildcard=True):
  
    """
    Identify column name from a given value using fuzzy search. 
    
    Parameters:
        ix (Index): The index to search in.
        keyword (str): The keyword to look for in the unique values of the columns.
        use_wildcard (bool): Whether to use wildcards or not. Default is True.
            
    Returns:
        dict: A dictionary where keys are column names and values are a list of unique values that match the keyword.
    """
    identified_column_dict = dict()
    with ix.searcher() as searcher:  
        ###Fuzzy lookup
        parser = QueryParser("unique_values", ix.schema)
        parser.add_plugin(FuzzyTermPlugin())
        if use_wildcard:
            query = parser.parse(f"*{keyword}*")  # Add wildcard characters to match any word that contains the keyword
        else:
            query = parser.parse(keyword)  
        ####
        results = searcher.search(query, limit=10)
        results.fragmenter.charlimit = None
        # Show more context before and after
        results.fragmenter.surround = 100

        for result in results:
            # print(result["column_name"], {result.score})
 
            # Remove '</b>' tags  
            search_term = result.highlights("unique_values", top=1).replace('</b>', '')  
            # Remove '<b class="match term[number]"' tags using regex  
            search_term = re.sub(r'<b class="match term\d+"', '', search_term)  
            # Split the text  
            search_term = search_term.split(", '") 
            # Remove char '>'
            search_term = [re.sub(r'>','',i) for i in search_term if '>' in i]
            # print(search_term)
            identified_column_dict[result["column_name"]] = search_term

    return identified_column_dict

identify_column_from_value(ix, "'Data Scientist'")

{}

In [53]:
import re
from whoosh.qparser import QueryParser, FuzzyTermPlugin

def identify_column(ix, keyword):
    """
    Detect column names and the synonyms that matched the keyword.
    
    Parameters:
        ix (Index): The index to search in.
        keyword (str): The keyword to look for in the synonyms of the columns.
            
    Returns:
        dict: A dictionary where keys are column names and values are lists of matched synonyms.
    """
    identified_column_dict = dict()

    with ix.searcher() as searcher:
        # Setup fuzzy lookup with wildcard
        parser = QueryParser("synonyms", ix.schema)
        parser.add_plugin(FuzzyTermPlugin())
        query = parser.parse(f"*{keyword}*")
        
        results = searcher.search(query, limit=30)
        results.fragmenter.charlimit = None
        results.fragmenter.surround = 100

        for result in results:
            column_name = result["column_name"]
            # Process search highlights to extract matched synonyms
            search_term_highlights = result.highlights("synonyms", top=1).replace('</b>', '')
            search_term_highlights = re.sub(r'<b class="match term\d+"', '', search_term_highlights)
            search_terms = search_term_highlights.split(", '")
            search_terms = [re.sub(r'>','',i) for i in search_terms if '>' in i]
            
            identified_column_dict[column_name] = search_terms

    return identified_column_dict

identify_column(ix, "cate")


{"'job_category', 'object'": ["job category'", "job categories'"],
 "'employment_type', 'object'": ['category']}

# Extract Keywords

In [37]:
# Read the file '.txt' and store it in a list with all words in lower case
with open(f'{DATA_PATH}/extra_words.txt') as f:
    money_words = f.read().splitlines()
money_words = [word.lower() for word in money_words]

def extract_keywords(text, money_words=[]):  
    '''Extract Keywords from the user query'''  
    is_date_present = False
    # Extract words inside single or double quotes  
    quoted_words = re.findall(r'"([^"]*)"|\'([^\']*)\'', text)  
    # Flatten the list of tuples and remove empty strings  
    quoted_words = [word for words in quoted_words for word in words if word]  
  
    # Remove quoted words from the original text  
    modified_text = re.sub(r'"[^"]*"|\'[^\']*\'', '', text)  
  
    # Proceed with the existing keyword extraction process  
    modified_text = modified_text.lower()  
    # Remove date related words
    pattern = r"\b(?:january|jan|february|feb|march|mar|april|apr|may|june|jun|july|jul|august|aug|september|sep|october|oct|november|nov|december|dec|20[0-4][0-9]|2050|quarter|year|month|ytd|mtd|today|yesterday|tomorrow|qtd)\b"  
    if re.search(pattern, modified_text, flags=re.IGNORECASE):  
        is_date_present = True

    modified_text = re.sub(pattern, "", modified_text, flags=re.IGNORECASE)  
    modified_text = re.sub(r'[^a-zA-Z0-9]', ' ', modified_text)  
    modified_text = modified_text.split()  
    modified_text = [word for word in modified_text if word not in money_words]  
    if is_date_present:
        modified_text.extend(['datetime64[', 'datetime32['])
    # Combine the quoted words and the extracted keywords  
    result = quoted_words + modified_text  
  
    return result, is_date_present  

question = "show all position positions?"
keywords, is_date_present = extract_keywords(question,money_words)
keywords, is_date_present

(['show', 'position', 'positions'], False)

In [38]:
def extract_columns(ix, keywords):
    """
    Extract columns based on given keywords by searching in attribute synonym knowledge base (kb).
    
    Parameters:
        ix (Index): The index to search in.
        keywords (list): A list of keywords to look for in the column names and synonyms.
            
    Returns:
        set, dict: A set of column names detected from the synonym kb and a dictionary mapping each column name to a list of keywords that were matched in it.
    """
    columns_detected = set()
    column_keywords_mapping = {}  # Dictionary to store which columns had which keywords detected

    for keyword in keywords:
        matched_columns = identify_column(ix, keyword)  # Returns a dictionary of column names to matched synonyms
        for item in matched_columns.items():
            columns_detected.add(item[0])  # Add the column to the set of detected columns
            if item[0] not in column_keywords_mapping:
                column_keywords_mapping[item[0]] = item[1]
            else:
                # Add the keyword if not already in the list for this column
                if keyword not in column_keywords_mapping[item[0]]:
                    column_keywords_mapping[item[0]].append(item[1])

    return columns_detected, column_keywords_mapping

columns_detected, keywords_matched_columns = extract_columns(ix,keywords)
print(keywords_matched_columns)

def extract_columns_from_values(ix, keywords):
    """
    Step 2: Looking in attribute synonym kb if keyword not found in synonym kb
    Extract columns based on given keywords by searching in attribute unique values knowledge base if keyword not found in synonym kb.
    
    Parameters:
        ix (Index): The index to search in.
        keywords (list): A list of keywords to look for in the unique values of the columns.
            
    Returns:
        set, dict: A set of column names detected from the unique values kb and a dictionary mapping each column name to a list of unique values that match any keyword.
    """
    keyword_column_mapping = dict()
    columns_detected_from_values = []
    for keyword in keywords:
        if True:  # keyword not in keywords_matched_columns:
            # keyword_column_mapping[keyword] = identify_column_from_value(keyword) # Which word occurs where and how
            temp_dict = identify_column_from_value(ix, keyword)
            for item in temp_dict.items():
                if item[0] not in keyword_column_mapping:
                    keyword_column_mapping[item[0]] = item[1]
                else:
                    keyword_column_mapping[item[0]].extend(item[1])
    columns_detected_from_values = list(keyword_column_mapping.keys())
    return set(columns_detected_from_values), keyword_column_mapping

columns_detected_from_values, keyword_column_mapping = extract_columns_from_values(ix,keywords)
columns_detected_from_values, keyword_column_mapping

{"'job_title', 'object'": ["position'"], "'job_category', 'object'": ["position'"], "'employment_type', 'object'": ["position\\\\_type'"]}


(set(), {})

In [39]:
def get_column_intersection_union(columns_detected,columns_detected_from_values, type='union'):
    '''Get Intersection or Union of Columns from Step 1 and Step 2'''
    if type == 'union':
        return columns_detected_from_values.union(columns_detected)
    else:
        return columns_detected_from_values.intersection(columns_detected)

column_union = get_column_intersection_union(columns_detected,columns_detected_from_values)
column_union

{"'employment_type', 'object'",
 "'job_category', 'object'",
 "'job_title', 'object'"}

# Dynamic Schema

In [40]:
def generate_table_schema(keyword_column_mapping, keywords_matched_columns, column_union):
    schema_string = ''
    for column in column_union:
        column_data_type = ''#columns_df[columns_df['COLUMN_NAME']==column]['DATA_TYPE'].values[0].upper()
        if column in keyword_column_mapping:
            schema_string += f"{column} -- has these unique values: {str(keyword_column_mapping[column])[1:-1]} \n"
        else:
            schema_string += f"{column} -- also referred to as: {str(keywords_matched_columns[column])[1:-1]}\n"
    return schema_string

schema_string = generate_table_schema(keyword_column_mapping, keywords_matched_columns, column_union)
print(schema_string)

'job_title', 'object' -- also referred to as: "position'"
'job_category', 'object' -- also referred to as: "position'"
'employment_type', 'object' -- also referred to as: "position\\\\_type'"



# Generate Index

In [42]:
# # Testing generate_index script
from generate_index import generate_index
generate_index(file_name='Fruit Prices.csv')

column_name: 'Commodity', 'object'
synonyms: ['Commodity', 'Commodities', 'Goods', 'Product', 'Merchandise', 'Stock', 'Asset', 'Tradeable', 'Item', 'Ware', 'CommodityItem', 'Cargo']
unique_values: ['Apples', 'Apples, applesauce', 'Apples, ready-to-drink', 'Apples, frozen concentrate', 'Apricots', 'Apricots, packed in juice', 'Apricots, packed in syrup or water', 'Bananas', 'Berries, mixed', 'Blackberries', 'Blueberries', 'Cantaloupe', 'Cherries', 'Cherries, packed in syrup or water', 'Clementines', 'Cranberries', 'Dates', 'Figs', 'Fruit cocktail, packed in juice', 'Fruit cocktail, packed in syrup or water', 'Grapefruit', 'Grapefruit, ready-to-drink', 'Grapes', 'Grapes (raisins)', 'Grapes, ready-to-drink', 'Grapes, frozen concentrate', 'Honeydew', 'Kiwi', 'Mangoes', 'Nectarines', 'Oranges', 'Oranges, ready-to-drink', 'Oranges, frozen concentrate', 'Papaya', 'Peaches', 'Peaches, packed in juice', 'Peaches, packed in syrup or water', 'Pears', 'Pears, packed in juice', 'Pears, packed in sy

In [ ]:
# Generate Context
import get_dynamic_schema
importlib.reload(get_dynamic_schema)
print(get_dynamic_schema.get_dynamic_schema(user_query="overall sales of envelope in 2022, xerox consumer standard second?", search_mode='strict',file_name="Global Superstore.csv"))

In [48]:
# Load Dataframe
import pandas as pd
file_name = 'jobs_in_data.csv'
df = pd.read_csv(file_name)
top_3_rows = str(df.head(3).to_dict())

In [50]:
import generate_prompt
importlib.reload(generate_prompt)

prompt, db_query, token_usage = generate_prompt.generate_prompt(
    user_query= "'Data Scientist''", 
    file_name=file_name, 
    topk_rows=top_3_rows, 
    search_mode='flexi'
)
print(prompt)


    ### Boss's Question:
    'Data Scientist''
    ### Some values and their respective columns (if detected) from the Question are:
    'job_title', 'object' -- has these unique values: "Data Analytics Specialist'", "Big Data Architect'", "Marketing Data Engineer'", "Manager Data Management'", "Data Analytics Consultant'", "Data Science Tech Lead'", "Data Scientist Lead'", "Marketing Data Analyst'", 'Data Analytics' 

    ## Instructions:
    It is in the format <column-name>, <data-type>, <unique-values>. Use the columns mentioned, their synonyms and it's unique values to frame your code in order to answer the question as correctly as possible.
    The dataframe in question has already been loaded as 'df', so you don't need to do it(i.e. never use the load data code).

    Always keep the python code(executable code only, no description) in this format: ```python <generated-code> ```
    There's no need to show the output or output might look because Boss will execute the generated 

In [106]:

import search_suggestions
importlib.reload(search_suggestions)
def get_suggestions(ix, user_query:str, enable_fuzzy:bool):
    """
    Get a list of sentence suggestions based on the user's query and index. If fuzzy search is enabled, use it to generate suggestions; otherwise, enable spell correction and fuzzy searching.

    Args:
    - ix (inverted index): The inverted index used for generating sentence suggestions.
    - user_query (str): The input user's query.
    - enable_fuzzy (bool): A flag to determine whether to use fuzzy search or not.

    Returns:
    - list of str: A list of suggested sentences based on the user's query and index, with a maximum length of 5.
    """

    if enable_fuzzy:
        suggested_sentences = search_suggestions.suggest_sentences(
            ix=ix, input_sentence=user_query, top_k=5
        )
    else:
        suggested_sentences = search_suggestions.suggest_sentences(
            ix=ix,
            input_sentence=user_query,
            top_k=5,
            enable_spell_correct=True,
            enable_fuzzy=enable_fuzzy,
        )
    return suggested_sentences

get_suggestions(ix, "what is united in-person ", enable_fuzzy=True)

Did you mean(identify_column): in-person?


["what is united 'In-person' <'work_setting'>'"]

In [27]:
def extract_keywords(text, money_words=[]):  
    '''Extract Keywords from the user query'''  

    # Load Keywords
    with open(f'extra_words.txt') as f:
        money_words = f.read().splitlines()
    money_words = [word.lower() for word in money_words]

    is_date_present = False
    # Extract words inside single or double quotes  
    quoted_words = re.findall(r'"([^"]*)"|\'([^\']*)\'', text)  
    # quoted_words = re.findall(r'"([^"]*)"|\'([^\']*)\'|\'\'([^\']*)\'\'', text)


    print(f"Quoted Words: {quoted_words}")

    # Flatten the list of tuples and remove empty strings  
    quoted_words = [word for words in quoted_words for word in words if word]  
  
    # Remove quoted words from the original text  
    modified_text = re.sub(r'"[^"]*"|\'[^\']*\'', '', text)  
    # modified_text = re.sub(r'"[^"]*"|\'[^\']*\'|\'\'[^\']*\'\'', '', text)

  
    # Proceed with the existing keyword extraction process  
    modified_text = modified_text.lower()  

    print(f"Modified Text: {modified_text}")

    # Remove date related words
    pattern = r"\b(?:january|jan|february|feb|march|mar|april|apr|may|june|jun|july|jul|august|aug|september|sep|october|oct|november|nov|december|dec|20[0-4][0-9]|2050|quarter|year|month|ytd|mtd|today|yesterday|tomorrow|qtd)\b"  
    if re.search(pattern, modified_text, flags=re.IGNORECASE):  
        is_date_present = True

    modified_text = re.sub(pattern, "", modified_text, flags=re.IGNORECASE)  
    modified_text = re.sub(r'[^a-zA-Z0-9]', ' ', modified_text)  
    modified_text = modified_text.split()  
    modified_text = [word for word in modified_text if word not in money_words]  
    if is_date_present:
        modified_text.extend(['datetime64[', 'datetime32['])
    # Combine the quoted words and the extracted keywords  
    result =  modified_text # + quoted_words 

    print(f"Extracted Keywords: {result}")
  
    return result, is_date_present  

result, is_date_present  = extract_keywords(f"""show all ''Commodity'' without 'Apples' <'Commodity'>' """.replace("''","'"))
result

Quoted Words: [('', 'Commodity'), ('', 'Apples'), ('', 'Commodity')]
Modified Text: show all  without  <>' 
Extracted Keywords: ['show']


['show']